In [6]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA

In [3]:
data = pd.read_csv('data/VARCTIC8.csv')

In [8]:
y_train = data['CO2_MaunaLoa'].iloc[:-100]
y_test = data['CO2_MaunaLoa'].iloc[-100:]

In [7]:
arima_model = ARIMA(y_train, order = (4, 0, 0))
results = arima_model.fit()

/home/isaac/.local/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/isaac/.local/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [12]:
def get_ar_forecasts(y_in, results_coefs, h):
  # y_in should be of length 'ar_lags' - all you need to start iterating forward
  
  # Store both train and test 
  y_all = np.zeros((y_in.shape[0] + h))
  y_all[:] = np.nan
  y_all[:y_in.shape[0]] = y_in

  for horizon in range(1, h+1):
    # Get the input lags for 
    y_in_this = y_all[(y_in.shape[0] -1 -4 + horizon):(y_in.shape[0] -1 + horizon)]
    # Evaluate the AR equation to get the prediction
    y_all[y_in.shape[0] - 1 + horizon] = results_coefs[0] + np.dot(results_coefs[1:], y_in)

  # Return self.h predictions
  return y_all[y_in.shape[0]:]

In [14]:
ar_lags = 4
reestimation_window = 100
test_size = 100
r = 0
h = 6

arima_model = ARIMA(y_train, order = (ar_lags,0,0))
results = arima_model.fit()
results_coefs = results.params[0:(ar_lags + 1)]

for t in range(0, 100):
  # Different cases of getting the input y_in
  if t == reestimation_window * r:
    y_in = y_train[-ar_lags:]
  # In between case when you need to combine Y_train and some bit of Y_test
  elif t < reestimation_window * r + ar_lags:
    train_obs_needed = ar_lags - (t - reestimation_window * r)
    test_obs_needed = t - reestimation_window * r
    y_in = np.concatenate([y_train[-train_obs_needed:], y_test[:test_obs_needed]], axis = 0)
  else:
    y_in = y_test[(t-ar_lags):t]

  fcast = get_ar_forecasts(y_in, results_coefs, h)
  print(f'{t} - y_in: {y_in}, fcast: {fcast}')

/home/isaac/.local/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


0 - y_in: 405    397.25
406    397.35
407    397.78
408    397.74
Name: CO2_MaunaLoa, dtype: float64, fcast: [762.89198456 762.89198456 762.89198456 762.89198456 762.89198456
 762.89198456]
1 - y_in: [397.35 397.78 397.74 397.45], fcast: [763.18410404 763.18410404 763.18410404 763.18410404 763.18410404
 763.18410404]
2 - y_in: [397.78 397.74 397.45 398.37], fcast: [763.317495 763.317495 763.317495 763.317495 763.317495 763.317495]
3 - y_in: [397.74 397.45 398.37 398.63], fcast: [763.16714633 763.16714633 763.16714633 763.16714633 763.16714633
 763.16714633]
4 - y_in: 409    397.45
410    398.37
411    398.63
412    398.56
Name: CO2_MaunaLoa, dtype: float64, fcast: [763.21926193 763.21926193 763.21926193 763.21926193 763.21926193
 763.21926193]
5 - y_in: 410    398.37
411    398.63
412    398.56
413    399.10
Name: CO2_MaunaLoa, dtype: float64, fcast: [763.98746993 763.98746993 763.98746993 763.98746993 763.98746993
 763.98746993]
6 - y_in: 411    398.63
412    398.56
413    399.10
414 